In [ ]:
# google colab の実行環境にデータをダウンロード (下の行のコメントアウトを外して実行してください)
# !wget https://raw.githubusercontent.com/KHiraGit/suds2503/refs/heads/main/london_merged.csv

In [ ]:
# 1. ライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
# 2. データセットの読み込み
url = "london_merged.csv"
df = pd.read_csv(url)

# 日時データの処理 (日付のデータ型を変更、時と曜日の列を作成)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['hour'] = df['timestamp'].dt.hour
df['day_of_week'] = df['timestamp'].dt.dayofweek
df = df.sort_values('timestamp')

In [ ]:
# データの概要(先頭)を表示
df.head()

In [ ]:
# データの概要(末尾)を表示
df.tail()

In [ ]:
# データの基本情報を表示
df.info()

In [ ]:
# データの欠損を確認
print(df.isnull().sum())

In [ ]:
# データの統計的概要を表示
df.describe()

In [ ]:
# 利用数の推移をプロット
plt.figure(figsize=(12, 6))
plt.bar(df.index, df['cnt'], color='skyblue')
plt.title('Bike Rentals in London')
plt.xlabel('Record Index')
plt.ylabel('Total Rentals')
plt.grid()
plt.show()

In [ ]:
# 日別の利用数の推移をプロット
plt.figure(figsize=(12, 6))
df_month = df.set_index('timestamp')['cnt'].resample('D').sum()
plt.bar(df_month.index, df_month.values, color='skyblue')
plt.title('Daily Bike Rentals in London')
plt.xlabel('Date')
plt.ylabel('Total Rentals')
plt.grid()
plt.show()

In [ ]:
# 月別の利用数の推移をプロット
plt.figure(figsize=(12, 6))
df_month = df.set_index('timestamp')['cnt'].resample('ME').sum()
plt.bar(df_month.index, df_month.values, width=20, color='skyblue')
plt.title('Monthly Bike Rentals in London')
plt.xlabel('Month')
plt.ylabel('Total Rentals')
plt.grid()
plt.show()

In [ ]:
# 時間ごとの利用数の推移をプロット
plt.figure(figsize=(12, 6))
df_hour = df.groupby('hour')['cnt'].mean()
plt.bar(df_hour.index, df_hour.values, color='orange')
plt.title('Average Bike Rentals by Hour of Day')
plt.xlabel('Hour of Day')
plt.ylabel('Average Rentals')
plt.grid()
plt.show()

In [ ]:
# 曜日ごとの利用数の推移をプロット
plt.figure(figsize=(12, 6))
df_hour = df.groupby('day_of_week')['cnt'].mean()
plt.bar(df_hour.index, df_hour.values, color='green')
plt.title('Average Bike Rentals by Day of Week (0=Monday, 6=Sunday)')
plt.xlabel('Day of Week')
plt.ylabel('Average Rentals')
plt.grid()
plt.show()

In [ ]:
# データの相関行列を作成して、ヒートマップで表示
corr_matrix = df.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix of Bike Rental Data')
plt.show()

In [ ]:
# 気温t1を5度ごとでcntを集計し、棒グラフで表示
plt.figure(figsize=(12, 5))
df_temp = df.groupby(pd.cut(df['t1'], bins=np.arange(-5, 40, 2.5)))['cnt'].mean()
df_temp.plot(kind='bar', color='lightgreen')
plt.title('Average Bike Rentals by Temperature (t1)')
plt.xlabel('Temperature (t1) [°C]')
plt.ylabel('Average Rentals')
plt.grid()
plt.show()

In [ ]:
# 湿度humを10%ごとにcntを集計し、棒グラフで表示
plt.figure(figsize=(12, 5))
df_temp = df.groupby(pd.cut(df['hum'], bins=np.arange(0, 100, 10)))['cnt'].mean()
df_temp.plot(kind='bar', color='lightgreen')
plt.title('Average Bike Rentals by Humidity (hum)')
plt.xlabel('Humidity (hum) [%]')
plt.ylabel('Average Rentals')
plt.grid()
plt.show()

In [ ]:
# weather_codeごとにcntを集計し、棒グラフで表示
# 天候コード 1=快晴 2=晴れ（雲が点在） 3=晴れ（切れ切れの雲） 4=曇り 7=雨/小雨/弱い雨 10=雷雨を伴う雨 26=降雪 94=凍結霧
plt.figure(figsize=(12, 5))
df_temp = df.groupby('weather_code')['cnt'].mean()
df_temp.plot(kind='bar', color='lightgreen')
plt.title('Average Bike Rentals by Weather Code')
plt.xlabel('Weather Code')
plt.ylabel('Average Rentals')
plt.grid()
plt.show()

In [ ]:
# weather_codeごとの利用パターンの違い
plt.figure(figsize=(12, 5))
sns.lineplot(data=df[df['weather_code'] == 1], x='hour', y='cnt', label='Clear')
sns.lineplot(data=df[df['weather_code'] == 2], x='hour', y='cnt', label='Sunny')
sns.lineplot(data=df[df['weather_code'] == 3], x='hour', y='cnt', label='Sunny with scattered clouds')
sns.lineplot(data=df[df['weather_code'] == 4], x='hour', y='cnt', label='Cloudy')
sns.lineplot(data=df[df['weather_code'] == 7], x='hour', y='cnt', label='Rain')
# sns.lineplot(data=df[df['weather_code'] == 10], x='hour', y='cnt', label='Thunderstorm')
# sns.lineplot(data=df[df['weather_code'] == 26], x='hour', y='cnt', label='Snow')
plt.title('Bike Rentals by Weather Code')
plt.grid(True)
plt.show()

In [ ]:
# is_weekendにcntを集計し、棒グラフで表示 (is_weekend 0=平日 1=週末)
plt.figure(figsize=(12, 5))
df_temp = df.groupby('is_weekend')['cnt'].mean()
df_temp.plot(kind='bar', color='lightgreen')
plt.title('Average Bike Rentals by Weekend Status')
plt.xlabel('Weekend Status (0=Weekday, 1=Weekend)')
plt.ylabel('Average Rentals')
plt.grid()
plt.show()

In [ ]:
# 平日と休日の利用パターンの違い
plt.figure(figsize=(12, 5))
sns.lineplot(data=df, x='hour', y='cnt', hue='is_weekend')
plt.title('Bike Rentals: Weekdays (0) vs Weekends (1)')
plt.grid(True)
plt.show()

In [ ]:
# is_holidayでcntを集計し、棒グラフで表示 (is_holiday 0=平日 1=祝日)
plt.figure(figsize=(12, 5))
df_temp = df.groupby('is_holiday')['cnt'].mean()
df_temp.plot(kind='bar', color='lightgreen')
plt.title('Average Bike Rentals by Holiday Status')
plt.xlabel('Holiday Status (0=Weekday, 1=Holiday)')
plt.ylabel('Average Rentals')
plt.grid()
plt.show()

In [ ]:
# 祝日の利用パターン
plt.figure(figsize=(12, 5))
sns.lineplot(data=df, x='hour', y='cnt', hue='is_holiday')
plt.title('Bike Rentals: Not Holiday (0) vs Holiday (1)')
plt.grid(True)
plt.show()

In [ ]:
# seasonでcntを集計し、棒グラフで表示
# season: 0=春　1=夏　2=秋　3=冬
plt.figure(figsize=(12, 5))
df_temp = df.groupby('season')['cnt'].mean()
df_temp.plot(kind='bar', color='lightgreen')
plt.title('Average Bike Rentals by Season')
plt.xlabel('Season (0=Spring, 1=Summer, 2=Autumn, 3=Winter)')
plt.ylabel('Average Rentals')
plt.grid()
plt.show()

In [ ]:
# 季節ごとの利用パターン
plt.figure(figsize=(12, 5))
sns.lineplot(data=df, x='hour', y='cnt', hue='season')
plt.title('Bike Rentals by Season')
plt.grid(True)
plt.show()

In [ ]:
# 利用数を予測するための線形モデルを作成し、精度を検証
df_tmp = df.copy()

# dfを学習用とテスト用に分割
# 特徴量としてt1（気温）、t2（体感温度）、hum（湿度）、wind_speed（風速）、is_weekend（週末かどうか）、is_holiday（祝日かどうか）、season（季節）を使用
X = df_tmp[['t1', 't2', 'hum', 'wind_speed', 'is_weekend', 'is_holiday','season']]
y = df_tmp['cnt'] # 目的変数
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 線形回帰モデルを作成
model = LinearRegression()
model.fit(X_train, y_train)
# テストデータで予測
y_pred = model.predict(X_test)
# 各種評価指標を計算
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'All Hours: Mean Absolute Error: {mae:.2f} Mean Squared Error: {mse:.2f} R2 Score: {r2:.2f}')

In [ ]:
# 各時刻の利用数を予測するための線形モデルを作成し、精度を検証
for _hour in range(24):
    # _hourのデータを抽出
    df_tmp = df.query(f'hour == {_hour}')

    # dfを学習用とテスト用に分割
    # 特徴量としてt1（気温）、t2（体感温度）、hum（湿度）、wind_speed（風速）、is_weekend（週末かどうか）、is_holiday（祝日かどうか）、season（季節）を使用
    X = df_tmp[['t1', 't2', 'hum', 'wind_speed', 'is_weekend', 'is_holiday','season']]
    y = df_tmp['cnt'] # 目的変数
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # 線形回帰モデルを作成
    model = LinearRegression()
    model.fit(X_train, y_train)
    # テストデータで予測
    y_pred = model.predict(X_test)
    # 各種評価指標を計算
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f'{_hour}: Mean Absolute Error: {mae:.2f} Mean Squared Error: {mse:.2f} R2 Score: {r2:.2f}')